In [1]:
from pathlib import Path 
from typing import *

import s3fs
import pandas as pd 
from sqlalchemy import select

from expression_atlas_db import base, load_db, settings, queries

In [30]:
def fetch_new_studies_qc(
    qc_loc: Path = Path(settings.s3_staging_loc),
) -> Tuple[pd.DataFrame, pd.DataFrame, int]:
    """Dumps a new qc template to qc_loc with current studies in db and their public/quality fields.

    Args:
        qc_loc (Path): Location in s3 for qc files (defaults to the staging_loc).
    Returns:
        (Tuple[pd.DataFrame, pd.DataFrame]): New studies, and the qc dataframe to be updated.
    """

    s3 = s3fs.S3FileSystem()

    qc_files = s3.glob(str(qc_loc / "qc*txt").replace("s3:/", "s3://"))
    qc_files = sorted(
        qc_files, key=lambda x: int(Path(x).parts[-1].split(".")[1]), reverse=True
    )

    with s3.open(qc_files[0], "rb") as f_in:
        qc_new_df = pd.read_csv(f_in, sep="|")
    new_studies_df = qc_new_df.copy()

    if len(qc_files) > 1:
        with s3.open(qc_files[1], "rb") as f_in:
            qc_old_df = pd.read_csv(f_in, sep="|")
        new_studies_df = qc_new_df.loc[
            ~qc_new_df['velia_id'].isin(qc_old_df['velia_id'])
        ]

    qc_number = int(qc_files[0].split(".")[1])
    
    return new_studies_df, qc_new_df, qc_number

def update_new_studies_qc(
    new_studies_df: pd.DataFrame,
    qc_df: pd.DataFrame,
    qc_number: int,
    qc_loc: Path = Path(settings.s3_staging_loc),
) -> None:
    """ Update rows in qc sheet and update in s3.
    Args:
        new_studies_df (pd.DataFrame): Updated rows in qc_df.
        qc_df (pd.DataFrame): Full qc sheet.
        qc_number (int): qc number to write qc sheet to. 
        qc_loc (Path): Location in s3 for qc files (defaults to the staging_loc).
    """

    s3 = s3fs.S3FileSystem()

    new_studies_df.set_index('velia_id', inplace=True)
    qc_df.set_index('velia_id', inplace=True)

    qc_df.loc[new_studies_df.index, ['public', 'quality']] = new_studies_df.loc[:,['public', 'quality']]

    with s3.open(
        str(Path(qc_loc) / f"qc.{qc_number}.txt").replace("s3:/", "s3://"), "w"
    ) as f_out:
        qc_df.reset_index(drop=False).to_csv(f_out, index=False, sep="|")

In [31]:
new_studies_df, qc_df, qc_no = fetch_new_studies_qc()

In [ ]:
# Edit quality and public columns in new_studies_df.

# new_studies_df.loc[:,'public'] = True

In [34]:
update_new_studies_qc(
    new_studies_df,
    qc_df,
    qc_no,
)

In [7]:
Session = base.configure(settings.db_connection_string)
session = Session()

In [35]:
# Run update, pulling qc from s3 staging and updating qc dependent on qc file.

load_db.update_studies_qc(
    connection_string=settings.db_connection_string,
)

INFO:root:Updating QC sheet.
INFO:root:Reading QC sheet: velia-piperuns-dev/expression_atlas/staging/qc.17.txt.
INFO:root:Updated studies with QC sheet: velia-piperuns-dev/expression_atlas/staging/qc.17.txt.


In [37]:
list(session.query(base.Study.velia_id).filter(base.Study.public == True))

[('SRP450756',),
 ('SRP237754',),
 ('GSE152991',),
 ('SRP460210',),
 ('GSE194331',),
 ('GSE206364',),
 ('GSE213001',),
 ('SRP439183',),
 ('ERP151282',),
 ('GSE110914',),
 ('SRP422703',),
 ('SRP437403',),
 ('SRP265346',),
 ('GSE122340',),
 ('GSE231692',),
 ('GSE123018',),
 ('GSE83687',),
 ('ERP105501',),
 ('ERP109255',),
 ('GSE130955',),
 ('ERP146053',),
 ('GSE143323',),
 ('SRP035988',),
 ('SRP042228',),
 ('GSE102371',),
 ('GSE157159',),
 ('SRP053101',),
 ('SRP245393',),
 ('GSE208041',),
 ('SRP065812',),
 ('SRP096757',),
 ('GSE162828',),
 ('GSE72509',),
 ('SRP113470',),
 ('GSE80183',),
 ('GSE89408',),
 ('SRP090849',),
 ('SRP186450',),
 ('SRP201604',),
 ('SRP129004',),
 ('SRP226115',),
 ('SRP132990',),
 ('SRP191103',),
 ('SRP237545',),
 ('SRP277091',),
 ('SRP217231',),
 ('SRP220377',),
 ('SRP288300',),
 ('SRP250210',),
 ('SRP155976',),
 ('SRP278883',),
 ('SRP287301',),
 ('SRP293215',),
 ('SRP223858',),
 ('SRP298703',),
 ('SRP303290',),
 ('SRP108496',),
 ('SRP021193',),
 ('GSE131705',),
 